# Analyse File

In [163]:
#General imports
import sys
import pandas as pd
import numpy as np
import seaborn as sns

#Project specific imports
sys.path.insert(0, r'..\Data')
from MyPostgreSQL import MyPostgreSQL


In [164]:
def getSMA(days, ticker):
    "Returns a Dataframe of the moving average of <days> (input value) samples on table <ticker> (input value)"
    MyClass = MyPostgreSQL()
    (conn, cursor) = MyClass.getConnCursor()

    query = f""" 
            SELECT date, adj_close, AVG(adj_close)
            OVER(
                ORDER BY date
                ROWS BETWEEN {days-1} PRECEDING AND CURRENT ROW
            ) AS SMA
            FROM {ticker};
            """
    
    cursor.execute(query)
    
    rows = cursor.fetchall()
    df = pd.DataFrame(rows, columns=['date', 'adj_close', f'MA{days}'])
    df.set_index('date', inplace=True)
    return df

number = 3
df = getSMA(number, 'AAPL')



Connecting to database server...
Connection established to:  ('PostgreSQL 14.0, compiled by Visual C++ build 1914, 64-bit',)
Constructor: The ticker is set to "AAPL" and absolut data path is "C:\Users\david\OneDrive\Code\StockAnalysis\Data\SP500\"
Disconnected: the connection is now closed.


In [165]:
def getIncrementOrDecline( ticker):
    "Returns a Dataframe of the with values 1 (price up), -1 (price down) or 0 (price unchanged) from prior trading day."
    MyClass = MyPostgreSQL()
    (conn, cursor) = MyClass.getConnCursor()

    query = f""" 
            WITH cte_adj_close_lag AS (
                SELECT *, 
                LAG(adj_close,1) OVER (
                    ORDER BY date
                ) AS previous_adj_close
                FROM {ticker}
            ),
            cte_percentage AS (
                SELECT *,
                COALESCE(ROUND( CAST( (adj_close - previous_adj_close)/previous_adj_close * 100 AS NUMERIC) ,2), 0) AS percentage_change
                FROM cte_adj_close_lag
            )
            SELECT date, adj_close, COALESCE(previous_adj_close, adj_close), (percentage_change>0)::int*1 - (percentage_change<0)::int*1 AS AD
            FROM cte_percentage;
            """

    cursor.execute(query)
    rows = cursor.fetchall()
    df = pd.DataFrame(rows, columns=['date', 'adj_close', 'previous_adj_close', 'AD'])
    df.set_index('date', inplace=True)
    
    return df

In [168]:
df_pre = getIncrementOrDecline('AAPL')
df_pre.head(30)

Connecting to database server...
Connection established to:  ('PostgreSQL 14.0, compiled by Visual C++ build 1914, 64-bit',)
Constructor: The ticker is set to "AAPL" and absolut data path is "C:\Users\david\OneDrive\Code\StockAnalysis\Data\SP500\"
Disconnected: the connection is now closed.


,adj_close,previous_adj_close,AD
date,,,
1990-01-02,1.075957,1.075957,0
1990-01-03,1.083178,1.075957,1
1990-01-04,1.086788,1.083178,1
1990-01-05,1.090399,1.086788,1
1990-01-08,1.097620,1.090399,1
1990-01-09,1.086788,1.097620,-1
1990-01-10,1.039851,1.086788,-1
1990-01-11,0.996524,1.039851,-1
1990-01-12,0.996524,0.996524,0


In [167]:
df_pre['AD'] = (df_pre['percentage'] > 0)*1 - (df_pre['percentage'] < 0)*1
df_pre.head(10)

KeyError: 'percentage'

In [ ]:
df_50 = getSMA(50, 'AAPL')
df_200 = getSMA(200, 'AAPL')

In [ ]:
sns.lineplot(x=df_50.index, y=df_50['MA50'])
sns.lineplot(x=df_50.index, y=df_50['adj_close'])

In [ ]:
query = """ 
        SELECT COUNT(*) as num
        FROM aapl;
        """

MyClass = MyPostgreSQL()
(conn, cursor) = MyClass.getConnCursor()
cursor.execute(query)
data = cursor.fetchone()
print( 'Data is ', data)